# Decision Trees

In this notebook we discuss another popular classification algorithm, decision trees. 

A decision tree is quite intuitive and a building block for an often used algorithm, random forests (more on those later). Trees can be made to perform both regression and classification tasks. We'll focus on classification in this notebook and if you're interested you can build a regression tree in the homework.

## What You'll Accomplish

<ul>
    <li>We'll introduce the decision tree algorithm with a dumb example,</li>
    <li>Talk about the decision tree loss functions,</li>
    <li>Show how sklearn fits a tree,</li>
    <li>You'll make a tree to predict heart disease,</li>
    <li>Touch on how sklearn handles categorical features and decision trees.</li>
</ul>



In [ ]:
## For data handling
import pandas as pd
import numpy as np

## For plotting
import matplotlib.pyplot as plt
import seaborn as sns

## This sets the plot style
## to have a grid on a white background
sns.set_style("whitegrid")

## The Basic Idea

The basic idea for decision trees is segmenting the data space in a way that allows us to classify the data well.

### A Dumb Example

Let's look at a really dumb example. Below we generate some random data with an $x_1$ feature, an $x_2$ feature, and a target, $y$. We then plot the data using seaborn. 

In [ ]:
df = pd.DataFrame({'x1':np.random.random(200),'x2':np.random.random(200),'y':np.zeros(200)})

df.loc[100:199,['x1']] = df.loc[100:199,['x1']] + 1
df.loc[100:199,['y']] = df.loc[100:199,['y']] + 1

In [ ]:
plt.figure(figsize = (10,10))
plt.scatter(df.loc[df.y == 0,'x1'],df.loc[df.y == 0,'x2'],c='orange',label="0")
plt.scatter(df.loc[df.y == 1,'x1'],df.loc[df.y == 1,'x2'],c='blue',label="1")
plt.xlabel("$x_1$",fontsize = 16)
plt.ylabel("$x_2$",fontsize = 16)
plt.legend(fontsize='14', title_fontsize='16')

plt.show()

Now if you had to come up with an adhoc classification rule right now what would you say we should do?

<br>
<br>
<br>

One good choice would be to say that if the data point is to the left of $x_1= 1$ then we classify it as $0$, and if it to the right of $x_1=1$ then we classify it as $1$. We can illustrate this with a logic tree
<img src = "SimpleTree.png" style="width:60%"></img>
While this is a very dumb example, it is the basic idea behind decision trees. You look at a feature, make a cut point that minimizes some measure of wrongness, and keep going until some stopping criterion.

Before we move on to real data, let's fit a decision tree to our dumb data and see what classification rule it comes up with.

In [ ]:
# Import packages
# tree will be used to plot the decisiton tree
from sklearn import tree

# This is the actual out of the box algorithm
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Make an decisiion tree object
tree_clf = DecisionTreeClassifier(random_state = 440)

In [ ]:
plt.figure(figsize = (10,10))
# Plot the fitted tree
fig = tree_clf.fit(df[['x1','x2']], df.y)
tree.plot_tree(fig,filled = True)
plt.show()


The plot above is the logic tree built by the decision tree algorithm. To classify a new observation we start at the <i>root node</i> up top. If the observation satisfies the logic statement at the top we go left and are classified as a $0$, else we go right and are classified as $1$. The two <i>children</i> of the root node are known as <i>leaf nodes</i> or <i>terminal nodes</i> because they have no children of their own so we just predict the majority class contained in that node.

This is essentially the decision rule we came up with (which is the objectively correct one by the way), so in this dummy example the decision tree did well.


If we look in the plot above we'll notice a number of different stats in each node. Let's quickly break those down. 

<ul>
    <li>samples - the number of samples in each node</li>
    <li>gini - the gini impurity of the node, more on this in a moment</li>
    <li>value - the breakdown of the number of samples of each target value in the node, for example the leaf node on the left has $100$ nodes labeled $0$ and $0$ nodes labeled $1$</li>
    <li>A decision rule - The rule that is used for the following split, samples that would be evaluated as True for the rule go to the left child, samples that would be evaluated as False go to the right child</li>
</ul>

### How is Wrongness Measured?

There are a couple of ways to measure wrongness, or rather impurity, with decision trees. We'll briefly touch on two popular measures that can implemented with out of the box `sklearn`.

#### Gini Impurity

Suppose that there are $N$ target classes.

The Gini Impurity for class $i$ of a node estimates the probability that a randomly chosen sample of class $i$ from the node is incorrectly classified as not class $i$. The formula is:
$$
G_i = p_i(1-p_i)
$$
where $p_i$ is the proportion of samples in the node of class $i$. The total Gini Impurity is the sum of all these $G_i$
$$
I_G = \sum_{i=1}^N G_i = 1 - \sum_{i=1}^N p_i^2
$$


#### Entropy

Entropy is an alternative impurity measure you could use when building a decision tree in `sklearn`. It's origins are in thermodynamics, but it soon moved into a diverse array of fields including information theory which is how it ended up in machine learning applications.

Again suppose there are $N$ target classes. The contribution made to entropy from class $i$ is:
$$
H_i = - p_i \log(p_i),
$$
where again $p_i$ is the proportion of samples in the node of class $i$. The total entropy of the node is the sum of all the $H_i$:
$$
I_H = \sum_{i=1}^N H_i = -\sum_{i=1}^N p_i \log(p_i) 
$$

#### Which to use

In most cases both measures are comparable, with Gini impurity being faster to compute (the $\log$ in entropy takes more time for a computer) it is a good default. It has been found that entropy leads to more balanced trees, https://sebastianraschka.com/faq/docs/decision-tree-binary.html, than Gini Impurity.

## How `sklearn` Fits a Tree

Now that we understand how to measure the impurity of a node, we can see how `sklearn` decides to make a node cut.

### The CART Algorithm

`sklearn` uses the <i>Classification and Regression Tree</i> or <i>CART</i> algorithm. 

Suppose your data set has $n$ observations with $m$ features, and for simplicity only $2$ target classes.

The algorithm starts with the root node. It then searches through each feature, $k$, and finds a split point, $t_k$, that produces the purest subsets (weighted by the number of samples in each subset), i.e. it finds a $t_k$ that minimizes:
$$
J(k,t_k) = \frac{n_\text{left}}{n} I_\text{left} + \frac{n_\text{right}}{n} I_\text{right},
$$
where left and right refers to being left or right of the split point, $t_k$, and $I$ is the impurity measure you choose to use (the default is Gini). 

Once it finds the $(k,t_k)$ pair that has smallest $J(k,t_k)$, it splits the data according to that decision split.

The algorithm then repeats the entire process on each of the children nodes it just produced. This continues until some stopping condition for example:
<ul>
    <li>reaching a maximum depth</li>
    <li>reaching a minimum number of samples in each node</li>
    <li>reaching a minimum weight to be in a node</li>
    <li>etc.</li>
</ul>
or until it can no longer reduce the impurity by making a cut.

## Quick Break

Okay before we move on to building a decision tree on real data, let's take a break for any questions about the algorithm, impurity, or anything we just went over.


Ask now.


Now let's try to build a model with non randomly generated data.

## Practice - Classifying Heart Disease with a Decision Tree


We'll be building a model to classify instances of heart disease from a data set that can be found <a href = "https://archive.ics.uci.edu/ml/datasets/heart+Disease">here</a>.  I walk you through some preliminary data exploration then leave you it to you to build the model.


### The Data


Before any data exploration let's make a test train split.

In [ ]:
# Import the data
# it is stored in the cleveland_heart csv
heart = pd.read_csv("cleveland_heart.csv")

# our target is whether or not there was heart disease
heart['target'] = 0
heart.loc[heart.status > 0,'target'] = 1


print("There are ",len(heart)," observations.")
print(sum(heart.target)," of those observations had heart disease")

In [ ]:
# Practice, make your train test split here!
# Keep the default of 25% set aside
# shuffle and stratify to ensure we keep the distr
# Set a random state of 614
from sklearn.model_selection import train_test_split

In [ ]:
# Use this block if needed
X_train,X_test,y_train,y_test = train_test_split(heart.drop(columns = ['status','target']),
                                                heart[['target']], 
                                                shuffle = True,
                                                stratify = heart[['target']], 
                                                random_state = 614)

X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

Okay now, using both the documentation and `pandas` explore the data set further before building a model. This will allow us to see if we need to do any cleaning before feeding the data into the algorithm.

Here are some questions you might want to know the answer to:
<ol>
    <li>Which variables are continuous? Categorical? Ordinal?</li>
    <li>What do each of the variables mean?</li>
    <li>How are the data values stored in the pandas dataframe? Is that appropriate for the data type?</li>
    <li>Do some variables appear to more important than others?</li>
</ol>

In [ ]:
X_train.head()


# from uci, the data source 
# age - ordinal
# sex - categorical/binary
# cp - categorical/1,2,3,4
# trestbps - cont.
# chol - cont
# fbs - cat/bin
# restecg - cat/0,1,2
# thalach - cont
# exang - cat/bin
# oldpeak - cont
# slope - ordinal
# ca - ordinal
# thal - cat/3,6,7

In [ ]:
X_train.info()

In [ ]:
#### Code here 
# We should recode the non-binary qualitative variables
X_train[['cp','restecg','thal']] = X_train[['cp','restecg','thal']].astype(str)
X_test[['cp','restecg','thal']] = X_test[['cp','restecg','thal']].astype(str)

In [ ]:
#### Code here 
# Let's examine the data by the target.
df_train = pd.concat([X_train,y_train],axis=1)

In [ ]:
sns.pairplot(df_train,vars = ['age','trestbps','chol','thalach','oldpeak'],
                  hue='target')

plt.show()

It looks like the continuous features may not be helpful on their own. Let's try to explore the categorical variables.

In [ ]:
df_train.head()

In [ ]:
for feature in ['sex','cp','fbs','restecg','exang','slope','ca','thal']:
    print(df_train.groupby(feature).target.value_counts(normalize=True).sort_index())
    print()

Okay now that we have a better feel for the data, let's build a decision tree.

### Non-binary Categorical Variables

If you read `sklearn`'s docs on the decision tree classifier they use, it does not support the use of categorical variables at the time of the writing of these notes.

This leaves two options:
<ol>
    <li>One hot encoding</li>
    <li>Removing these variables from the algorithm</li>
</ol>

We'll do both here. However, in a later notebook we'll return to this issue and see if we should be okay with building tree on one hot encoded variables in practice.

#### Model Without Non-Binary Categorical Variables

Build a model based on the variables that aren't categorical with more than two categories.

Produce an idea of how good your model is based on its:
<ul>
    <li>accuracy</li>
    <li>precision</li>
    <li>recall</li>
</ul>


Play around with the hyper parameters of max_depth and min_samples_split.

##### Caution

You should use either cross validation or set aside a small chunk of data within the training set to test your trees on. This is because decision trees can be made to fit the data perfectly if they're allowed to get deep enough. While this is great for understanding the training data, it's definitely overfitting. 



In [ ]:
## Code here
# Making functions to calculate various stats
## Accuracy
def get_acc(prediction,y):
    return np.round(np.sum(prediction == y)/len(y),5)*100
## Precision
def get_prec(prediction,y):
    positives = prediction[prediction == 1]
    return np.round(np.sum(positives == y[prediction == 1])/len(positives),5)*100
## Recall
def get_recall(prediction,y):
    ones = y[y==1]
    return np.round(np.sum(prediction[y==1] == ones)/len(ones),5)*100


In [ ]:
# Code here
# note StratifiedKFold performs stratified splits in cv
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone







In [ ]:
# Code Here








In [ ]:
# Code Here












In [ ]:
# Code Here












#### Model With Non-Binary Categorical Variables

Perform one hot encoding of the non-binary categorical variables, then repeat the prior steps. Is this model better on the test data than the last one?

Which measures should we use to evaluate better in this problem?

In [ ]:
## Code here
#










In [ ]:
## Code here













In [ ]:
## Code here















In [ ]:
## Code here
















### Write a summary of your findings here 










## Final Thoughts on Decision Trees

Before we close the jupyter notebook on decision trees let's leave with a few take aways.

### Advantages
<ol>
    <li>Interpret-ability - Decision Trees are known as a white box algorithm (as opposed to the black box often used to describe machine learning). This is because you are able to entirely describe how a decision tree predicts a data points target using the logic tree.</li>
    <li>Very Fast Predictions</li>
    <li>Very little preprocessing of data prior to training</li>
</ol>

### Disadvantages
<ol>
    <li>Greediness - The algorithm is greedy meaning it may not create the optimal tree. For example, maybe the best tree involves an initial suboptimal cut, the CART algorithm won't find this tree.</li>
    <li>Overfitting - Decision trees are very prone to overfitting the data, you can control for this using regularization hyperparameters like max_depth and min_samples_split. It's also a good idea to use cv when you can.</li>
    <li>Orthogonal Boundaries - Because of the process of determining cut points (remember the $t_k$ from the algorithm?) decision boundaries happen at right angles. This means that classes divided by a non-horizontal or non-vertical line the decision tree will have some capturing the boundary. This can be mitigated a bit with PCA.</li>
    <li>Sensitive - Trees are very sensitive to the training data. Removing or adding a few points can greatly change the decision boundary produced by the algorithm. One way around this is to use an averaged algorithm, like a random forest. We discuss these in the next notebook.</li>
</ol>

### Getting class probabilities

You can get class probabilities from a fitted decision tree classifier using the method `predict_proba(X)` where $X$ is the feature array you want to predict on.

These probabilities can then be used to calculate AUC for hyperparameter tuning.

In [ ]:
# An example of getting tree probabilities

# Make the classifier 
tree_clf = DecisionTreeClassifier(random_state = 440,max_depth = 3)

# Fit the classifier
tree_clf.fit(X_train,y_train)

# Get the probabilities for the training set
tree_clf.predict_proba(X_train)